# Master Functions - Annihilation Rate

## Import Other Notebooks

In [ ]:
import io, os, sys, types

from IPython import get_ipython
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell

def find_notebook(fullname, path=None):
    """find a notebook, given its fully qualified name and an optional path

    This turns "foo.bar" into "foo/bar.ipynb"
    and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
    does not exist.
    """
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path
        
class NotebookLoader(object):
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path

    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)

        print ("importing Jupyter notebook from %s" % path)

        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = read(f, 4)


        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod

        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__

        try:
            for cell in nb.cells:
                if cell.cell_type == 'code':
                    # transform the input to executable Python
                    code = self.shell.input_transformer_manager.transform_cell(cell.source)
                    # run the code in themodule
                    exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod

    
class NotebookFinder(object):
    """Module finder that locates Jupyter Notebooks"""
    def __init__(self):
        self.loaders = {}

    def find_module(self, fullname, path=None):
        nb_path = find_notebook(fullname, path)
        if not nb_path:
            return

        key = path
        if path:
            # lists aren't hashable
            key = os.path.sep.join(path)

        if key not in self.loaders:
            self.loaders[key] = NotebookLoader(path)
        return self.loaders[key]
    
sys.meta_path.append(NotebookFinder())



In [ ]:
from MasterFunctions_ModelData import *

-------------------

# Annihilation Functions

## $V_0$ Function

In [ ]:
def v0func(m_X):
    return np.sqrt(2*TCross/m_X)


## Annihilation Tree Cross Section

### Dark Photon

In [ ]:
def photonSigmaVtree(m_X, m_A, alpha_X): 
    '''
    sigmaVtree(m_X, m_A, alpha_X)
    '''
    numerator = (1 - (m_A/m_X)**2)**1.5
    denominator = ( 1 - 0.5 * (m_A/m_X)**2 )**2
    prefactor = np.pi*(alpha_X/m_X)**2
    
    function = prefactor * numerator/denominator
    return function


### (INCOMPLETE) Higgs

In [ ]:
# def higgsSigmaVtree():
#     return

## Sommerfeld Enhancement

In [ ]:
def sommerfeld(v, m_X, m_A, alpha_X):
    a = v / (2 * alpha_X)
    c = 6 * alpha_X * m_X / (np.pi**2 * m_A)
    
    # Kludge: Absolute value the argument of the square root inside Cos(...)
    function = np.pi/a * np.sinh(2*np.pi*a*c) / \
        ( np.cosh(2*np.pi*a*c) - np.cos(2*np.pi*np.abs(np.sqrt(np.abs(c-(a*c)**2)) ) ) )
    return function


## Thermal Average Sommerfeld

In [ ]:
def thermAvgSommerfeld(m_X, m_A, alpha_X):
    v0 = v0func(m_X)
    
    def integrand(v):
        # We perform d^3v in spherical velocity space.
        # d^3v = v^2 dv * d(Omega)
        prefactor = 4*np.pi/(2*np.pi*v0**2)**(1.5)
        function = prefactor * v**2 * np.exp(-0.5*(v/v0)**2) * sommerfeld(v, m_X, m_A, alpha_X)
        return function
    
    lowV = 0
    # Python doesn't like it when you integrate to infinity, so we integrate out to 10 standard deviations
    highV = 10*(v0func(m_X))

    integral = integrate.quad(integrand, lowV, highV)[0]
    return integral


## Annihilation Rate

In [ ]:
def CAnnCalc(m_X, sigmaVTree, thermAvgSomm = 1):
    '''
    CAnnCalc(m_X, sigmaVTree, thermAvgSomm = 1) by default, returns the Annihilation rate WITHOUT
    Sommerfeld effects. sigmaVTree and the sommerfeld enhancement contain the mediator mass dependence
    so this function CAnnCalc does not explicitly depend on the mediator mass.
    '''
    prefactor = (Gnat * m_X * rhoCross/ (3 * TCross) )**(3./2)
    conversion = (1.52e24) # GeV -> Sec^-1
    function = conversion * prefactor * sigmaVTree * thermAvgSomm
    return function


## Notebook Imported Message

In [ ]:
print ('------ MasterFunctions_Annihilation Imported ------')